In [1]:
import pandas as pd
import numpy as np
import findspark
import pyspark
from pyspark import SparkContext
from pyspark.sql.session import SparkSession
from pyspark import SparkConf

In [2]:
raw_data = spark.read.load("../data/raw_data/raw_data.csv",format="csv", sep=",", inferSchema="true", header="true")
print(raw_data.show(2))

+-------------+---+------+-----------------------+---+-----------+-------------------------------+----------------------+
|Child Product|CPG| Month|Shipment in Child Cases|PPG|Parent Prod| Shipment in parent Prod Cases |Inventory_Parent_Cases|
+-------------+---+------+-----------------------+---+-----------+-------------------------------+----------------------+
|           S1| C5|201801|                     26| P1|         S1|                             26|                     0|
|           S2| C5|201801|                     33| P1|         S1|                             25|                     0|
+-------------+---+------+-----------------------+---+-----------+-------------------------------+----------------------+
only showing top 2 rows

None


In [4]:
raw_data.limit(2).toPandas()

,Child Product,CPG,Month,Shipment in Child Cases,PPG,Parent Prod,Shipment in parent Prod Cases,Inventory_Parent_Cases
0,S1,C5,201801,26,P1,S1,26,0
1,S2,C5,201801,33,P1,S1,25,0
